금융 시계열 데이터를 대상으로 한 <strong>Classification 문제에서 합성곱 신경망(CNN)</strong>을 활용하는 것은 비교적 최근에 각광받고 있는 접근 방식입니다. 원래 CNN은 이미지 분석에 특화되어 있었지만, **1차원 CNN (1D-CNN)** 구조를 통해 **시계열 데이터 분석**에서도 매우 효과적인 분류 성능을 보입니다. 아래에서 금융 시계열 데이터에 특화된 관점에서 **모델 구조, 적용 방식, 장단점, 예시** 등을 자세히 설명드리겠습니다.

---

# 🧭 금융 시계열 Classification에서 CNN의 개요

## 🧠 문제 정의 예시

금융 시계열 Classification 문제란 예를 들어 다음과 같은 작업입니다:

* **내일 주가가 상승할지/하락할지를 예측** (이진 분류)
* **시장 상태를 'bull', 'bear', 'neutral'로 분류** (다중 분류)
* **트레이딩 신호 발생 여부 분류 (buy / hold / sell)**

이때 입력은 일반적으로 다음과 같은 형태입니다:

\| 시간      | 가격 | 거래량 | 지표1 | 지표2 | ... |
\|-----------|------|--------|-------|-------|
\| t-9       |      |        |       |       |
\| t-8       |      |        |       |       |
\| ...       |      |        |       |       |
\| t (now)   |      |        |       |       |

이러한 \*\*시계열 블록을 하나의 "입력 샘플"\*\*로 간주하여 CNN에 입력합니다.

---

# 🧱 CNN 구조 (1D-CNN 중심)

## 🧩 기본 구성

### 1. **1D Convolution Layer**

* 금융 시계열은 보통 다중 feature를 가진 1차원 배열 형태입니다.
* CNN은 이 1D 배열을 **슬라이딩 필터**로 이동하면서 **국소 패턴 추출**을 수행합니다.
* 예를 들어, “5일 간의 상승-하락 패턴” 같은 신호를 인식 가능

### 2. **Activation (ReLU)**

* 비선형성을 추가하여 복잡한 패턴 감지

### 3. **1D Pooling Layer (MaxPooling)**

* 정보 요약 및 다운샘플링 → 노이즈 제거, 시간 민감도 향상

### 4. **Flatten + Fully Connected Layer**

* 추출된 특징을 기반으로 마지막 분류기 (Dense layer + softmax 또는 sigmoid) 수행

---

## 🧪 Python 예시 (Keras 기준)

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential([
    Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(10, 5)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(3, activation='softmax')  # 예: 3-class 분류 (상승/보합/하락)
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
```

* `input_shape=(10, 5)`는 `10일 시계열`, `5개 특성` (예: 가격, 이동평균, RSI 등)을 의미

---

# ✅ CNN이 금융 시계열 분류에 적합한 이유

| 특징              | 설명                                    |
| --------------- | ------------------------------------- |
| **로컬 패턴 탐지**    | 국소적 가격 변화 패턴(예: 상승장 직전 특징)을 필터로 자동 인식 |
| **스무딩 효과**      | 잡음을 줄이면서 의미 있는 움직임만 추출                |
| **특징 추출 자동화**   | RSI, MACD 같은 전통적 지표 없이도 유효한 패턴을 학습 가능 |
| **다중 피처 조합 학습** | 가격 + 거래량 + 기술 지표 등 복합적인 상호작용 학습 가능    |

---

# ⚠️ 한계점 및 고려사항

| 항목              | 설명                                             |
| --------------- | ---------------------------------------------- |
| **해석력 부족**      | CNN 내부는 블랙박스화되므로 예측의 이유 해석이 어려움                |
| **시계열 길이 민감**   | 지나치게 짧거나 긴 시퀀스는 과적합/과소학습 발생 가능                 |
| **시간 지연 반영 한계** | 장기적 구조는 LSTM/Transformer 등 RNN 계열이 더 적합한 경우 있음 |
| **클래스 불균형**     | 상승/하락 외 대부분은 보합 → 클래스 재조정/샘플링 필요               |

---

# 🔁 CNN vs RNN vs 전통 모델

| 모델                    | 특징                     | 적합 용도                  |
| --------------------- | ---------------------- | ---------------------- |
| **CNN (1D)**          | 빠름, 로컬 패턴 학습, 병렬 처리 강점 | **단기 패턴 탐지**, 고주기 트레이딩 |
| **LSTM/GRU**          | 장기 의존성 학습, 순서 민감       | 추세 분석, 장기 가격 움직임 예측    |
| **SVM/Random Forest** | 해석력 높음, 정형 분류          | 단기 전략 모델, 인터프리터 필요 상황  |

➡ 실제 실무에서는 CNN과 LSTM을 **혼합(CNN-LSTM)** 하기도 합니다.

---

# 📊 성능 평가 지표 (금융에 특화)

* Accuracy는 의미가 없을 수 있음 → **Precision, Recall, F1-score** 중요
* **Profit-based Metrics**:

  * 총 수익률 (Total return)
  * MDD (Maximum Drawdown)
  * Win ratio
  * Sharpe ratio

---

# 🧩 결론

**금융 시계열 Classification에서 CNN은:**

> ✅ 정해진 길이의 짧은 시계열 내에서 국소 패턴(상승, 급락 등)을 자동으로 추출하고
> ✅ 여러 금융 지표나 입력 피처를 통합해 학습할 수 있는
> ✅ 매우 빠르고 강력한 분류 알고리즘입니다.

> 단기 전략 모델링, 트레이딩 신호 탐지, 가격 반응 분류 등에 매우 유용하며,
> 해석력과 시계열 길이 제약은 보완 고려 대상입니다.

---


* CNN과 LSTM을 결합한 하이브리드 모델
* 금융 데이터셋 (예: S\&P 500, 비트코인) 적용 코드
* 모델 explainability (예: Grad-CAM 등 시각화) 예시

